In [54]:
import pandas as pd
import openpyxl as xl
from shutil import copy
from openpyxl.styles import Alignment

Staff = pd.read_excel('Штат.xlsx')  # считываем файл со штатом сотрудников = Staff


Staff['full_name_dep'] = (Staff["Фамилия"] + " " + Staff["Имя"] + " " + Staff["Отчество"] 
                          + " (" + Staff["Штатная должность"]+")").astype("string") 
#делаем full_name - ФИО + департамент где работает



departments = Staff['Подразделение (наименование)'].unique() 
#находим все уникальные департменты в столбце "Подразделение (наименование)"


In [55]:
# можно здесь что-то подправить, если шаблон измениться 
template_size = 50 # ppl per file (кол-во людей, которое может быть в одном файле, если окажется что в департменте работает больше, просто изменить значение )
first_row = 18     # first row of the empty list in the template

#здесь как раз и отлавливаю ошибку, если вдруг окажется что лист очень маленький, а народу в департменте много
max_people = Staff.groupby('Подразделение (наименование)')['full_name_dep'].count().max()
if template_size < max_people:
    raise ValueError('The template is too small!')

last_row = first_row + template_size * 4 - 1  # последняя пустая строчка в  template

In [56]:
for department in departments:

    #создаем файл для каждого департмента 
    file = f'{department}.xlsx'
    copy('TableForm.xlsx', file) #название можно поменять
    wb = xl.load_workbook(file)
    sheet = wb['Табель'] #название листа, можно поменять на нужный в шаблоне на входе (то есть в TableForm.xlsx )
   
    top_left_cell = sheet['B5']
    top_left_cell.value = department
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")    
    #добавляем название департмента в центр над "структурное подразделение"
    
    
    
    

    

    # добавляем всех сотрудников в файл
    people = Staff.loc[Staff["Подразделение (наименование)"] == department,                # фильтр по отделу
                       ['full_name_dep', 'Табельный номер']] # столбцы которые нам надо
    people = people.sort_values('full_name_dep')
    n = people.shape[0] # n = кол-во людей 
    r = first_row       # r = текущая строчка 
    
    numbering = 1 # нумерация людей в списке 
    for person in people.itertuples():
        sheet.cell(r, 3).value = person[1]   # ФИО + должность 
        sheet.cell(r, 4).value = person[2]   # Табельный номер
        sheet.cell(r, 2).value = numbering
        numbering+=1
        
        r += 4 #добавляем через 4 клетки нового сотрудника 
    
    # удаляем все слитые НЕИСПОЛЬЗОВАННЫЕ клетки экселя чтобы они были не создавали проблем после добавления людей в файл департмента
    #P.S. раньше происходил непонятный сдвиг линий контура, удалялись которые слова и т.п.
    
    row = r #возвращеем row в изначальное состояние, чтобы сделать тотже (почти) цикл с другими объектами
    
    for row in range(row, last_row, 4):
        for col in [(2,2), (3,3), (4,4)]:
            sheet.unmerge_cells(start_row=row, start_column=col[0], end_row=row+3, end_column=col[1])
        #разъединение ячеек для избежание проблемы (слитые ячейки накладывались поверх подписей и прочего того, что снизу таблицы)
    row = r     #возвращеем row в изначальное состояние, чтобы сделать тотже (почти) цикл с другими объектами
    
    for row in range(row, last_row, 2):        
        for col in [(22,22)]:
            sheet.unmerge_cells(start_row=row, start_column=col[0], end_row=row+1, end_column=col[1])
        #разъединение ячеек для избежание проблемы (слитые ячейки накладывались поверх подписей и прочего того, что снизу таблицы)    
   
    row = r #возвращеем row в изначальное состояние, чтобы сделать тотже (почти) цикл с другими объектами
    
    
    for col in [(2,3), (21,22)]:
        sheet.merge_cells(start_row=row+1, start_column=col[0], end_row=row+1, end_column=col[1])
        #объединение ячеек для "Ответственное лицо" и "Руководитель структурного подразделения"

    
    row = r + 3 #возвращеем row в изначальное состояние, чтобы сделать тотже (почти) цикл с другими объектами
    
    for col in [(21,22)]:
        sheet.merge_cells(start_row=row+1, start_column=col[0], end_row=row+1, end_column=col[1]) 
        #объединение ячеек для "Работник кадровой службы"
        
#ЕСЛИ НУЖНО ПОНЯТЬ КАК РАБОТАЕТ РАЗЪЕИНЕНИЕ ЯЧЕЕК, ТО ВСТАВЬТЕ ЭТОТ PRINT СРАЗУ ПОСЛЕ ( for col in[])
#print("start_row = row = ",row,"//end_row = ",row+1, "//last_row =", last_row, "//first_row = ", first_row, "//r = ", r, "//col[0] = ", col[0], "//col[1] = ", col[1])
        
    row = r + 2
    for col in [(4,5), (7,9), (11,19), (23,24),(28,29)]:
        sheet.merge_cells(start_row=row, start_column=col[0], end_row=row, end_column=col[1])
        #соединяем ячейки для подписей по типу "дожность", "личная подпись и т.п."в первой строчке после "Отвеств. лицо"
        
    row = r + 5  #возвращеем row в изначальное состояние, чтобы сделать тотже (почти) цикл с другими объектами  
    
    
    for col in [(23,24),(28,29)]:
        sheet.merge_cells(start_row=row, start_column=col[0], end_row=row, end_column=col[1])
        #соединяем ячейки для подписей по типу "дожность", "личная подпись и т.п."в первой строчке после "Отвеств. лицо"
        
        
    rd = sheet.row_dimensions[r+1]   #расширяем ячейку для "Ответственное лицо" и "Руководитель структурного подразделения"
    rd.height = 33                   #в высоту
    rd1 = sheet.row_dimensions[r+4]  #расширяем ячейку для "Работник кадровой службы"
    rd1.height = 33                  #в высоту
            
        
    
    # удаляем неиспользованные строки 
    sheet.delete_rows(r, last_row + 1  - r)
    
    #корректируем print area чтобы она соотвестовала финальному результату, который нам нужен (без пустых пропусков и т.п)
    sheet.print_area = sheet.calculate_dimension()
    
    
    
    
    wb.save(file)   #сохраняем все изменения 
    
    # нужен для теста (сделает один файл), удалите break если надо чтобы сделало все департаменты (весь цикл)
    break
    